In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

In [2]:
# StandarizedRunway32 | PCARunway32 | StandarizedRunway18  | PCARunway18
dataset = 'PCARunway18.csv'

In [3]:
# Path con los datasets iniciales
#os.chdir('D:/Documents/UPM/Actual/TFG/LEMD/DataSets')
#os.chdir('C:/Users/aalfaro/Documents/Meteo/TFG/LEMD/DataSets')

os.chdir('/home/crida/notebooks/Meteo TFG Adrian/Datasets')

#pd.options.display.max_rows = 200
#pd.options.display.max_columns = 50

#pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [4]:
df = pd.read_csv(dataset, header = 0, decimal = ',', sep = ';')
df

,threshold_separation,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
0,7.106423,-2.173258,0.344812,1.714468,-0.488184,-0.914801,-1.031175,0.585364,-0.792150,0.562869,0.250876
1,8.360370,-2.194871,0.381794,1.973512,-0.718297,-0.554071,-0.803732,0.303407,1.942153,1.183070,0.827862
2,8.460610,-2.293616,0.376733,2.012047,-0.331183,-0.529994,-0.723533,0.287288,-0.960162,1.119520,0.240661
3,10.801400,-2.243287,0.357465,1.913242,-0.449455,-0.737973,-0.987928,0.555140,-0.485039,0.417554,0.178782
4,3.935571,-2.296365,0.411195,2.229333,-0.504964,0.015727,-0.213067,-0.292003,1.096390,2.198394,0.967981
...,...,...,...,...,...,...,...,...,...,...,...
22717,9.786464,2.304257,-1.786522,0.867814,-0.348051,2.114663,-3.375228,1.983801,-0.329355,-0.100990,2.811164
22718,12.601120,2.127268,-1.761390,1.458409,-0.090432,3.352354,-2.395057,1.008888,-1.086958,0.297370,3.005814
22719,6.150367,2.336759,-1.764126,0.613482,-0.264183,1.684800,-3.443336,2.034830,-1.049064,1.320771,2.853402
22720,8.067130,2.297790,-1.785664,0.882055,-0.367795,2.049742,-3.474159,2.087708,-0.093161,-0.260945,2.758780


In [5]:
X = df.drop(['threshold_separation'], axis=1)
y = df['threshold_separation']

In [6]:
# Regresion lineal
lr = LinearRegression()
params = {
    'fit_intercept': [True]
}

model = GridSearchCV(
    estimator = lr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=LinearRegression(), n_jobs=5,
             param_grid={'fit_intercept': [True]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [7]:
a = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

a = a.sort_values(by = ['Rank', 'Score Time'], ascending=True)
a.to_csv('Results/18PCA-LR.csv', decimal = ',', sep = ';', index = False)

a

,fit_intercept,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
0,True,1,0.006327,0.002749,-2.559581,0.009936,0.011102,-0.090686,-0.873577


In [8]:
# KNN
knn = KNeighborsRegressor()
params = {
    'n_neighbors': [10, 20, 30, 40],
    'weights': ['distance', 'uniform'],
    'metric': ['minkowski'], 
    'p': [2]
}

model = GridSearchCV(
    estimator = knn,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=5,
             param_grid={'metric': ['minkowski'],
                         'n_neighbors': [10, 20, 30, 40], 'p': [2],
                         'weights': ['distance', 'uniform']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [9]:
b = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

b = b.sort_values(by = ['Rank', 'Score Time'], ascending=True)
b.to_csv('Results/18PCA-KNN.csv', decimal = ',', sep = ';', index = False)

b

,metric,n_neighbors,p,weights,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
6,minkowski,40,2,distance,1,0.021593,0.363653,-2.533703,0.029709,0.032081,-0.089195,-0.854321
7,minkowski,40,2,uniform,2,0.019601,0.299826,-2.533994,0.029489,0.031913,-0.089315,-0.854702
5,minkowski,30,2,uniform,3,0.021325,0.338839,-2.541477,0.023777,0.026283,-0.089772,-0.859913
4,minkowski,30,2,distance,4,0.019722,0.308613,-2.542038,0.023325,0.025761,-0.089711,-0.860082
3,minkowski,20,2,uniform,5,0.020276,0.261365,-2.560771,0.008637,0.010937,-0.091023,-0.873389
2,minkowski,20,2,distance,6,0.021604,0.289620,-2.562699,0.007136,0.009395,-0.091076,-0.874481
1,minkowski,10,2,uniform,7,0.021532,0.236502,-2.624390,-0.041448,-0.039440,-0.095653,-0.919282
0,minkowski,10,2,distance,8,0.043603,0.255606,-2.627707,-0.044121,-0.042122,-0.095811,-0.921308


In [10]:
# Random Forest Regression
rfr = RandomForestRegressor(random_state = 1)
params = {
    'n_estimators': [100, 150, 200],
    'criterion': ['mse'],
    'max_depth': [None, 10, 12],
    'max_leaf_nodes': [None],
    'bootstrap': [True],
    'max_features': [0.5],
    'min_samples_split': [10, 15], 
    'min_samples_leaf': [5, 10]
}

model = GridSearchCV(
    estimator = rfr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 5,
    n_jobs = 5
)

model.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=1), n_jobs=5,
             param_grid={'bootstrap': [True], 'criterion': ['mse'],
                         'max_depth': [None, 10, 12], 'max_features': [0.5],
                         'max_leaf_nodes': [None], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15],
                         'n_estimators': [100, 150, 200]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [11]:
c = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

c = c.sort_values(by = ['Rank', 'Score Time'], ascending=True)
c.to_csv('Results/18PCA-RFR.csv', decimal = ',', sep = ';', index = False)

c

,bootstrap,criterion,max_depth,max_features,max_leaf_nodes,min_samples_leaf,min_samples_split,n_estimators,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
35,True,mse,12.0,0.5,None,10,15,200,1,8.743487,0.063149,-2.468559,0.078811,0.079196,-0.082180,-0.805320
32,True,mse,12.0,0.5,None,10,10,200,1,9.916977,0.070746,-2.468559,0.078811,0.079196,-0.082180,-0.805320
34,True,mse,12.0,0.5,None,10,15,150,3,7.431643,0.053691,-2.468678,0.078724,0.079126,-0.082202,-0.805427
31,True,mse,12.0,0.5,None,10,10,150,3,7.430595,0.054442,-2.468678,0.078724,0.079126,-0.082202,-0.805427
23,True,mse,10.0,0.5,None,10,15,200,5,8.879611,0.062103,-2.468679,0.078801,0.079134,-0.082226,-0.805629
20,True,mse,10.0,0.5,None,10,10,200,5,8.902072,0.062123,-2.468679,0.078801,0.079134,-0.082226,-0.805629
19,True,mse,10.0,0.5,None,10,10,150,7,6.657087,0.047219,-2.468883,0.078639,0.078975,-0.082250,-0.805794
22,True,mse,10.0,0.5,None,10,15,150,7,6.659131,0.047368,-2.468883,0.078639,0.078975,-0.082250,-0.805794
17,True,mse,10.0,0.5,None,5,15,200,9,9.004367,0.061136,-2.469462,0.078199,0.078680,-0.082422,-0.806062
16,True,mse,10.0,0.5,None,5,15,150,10,6.752052,0.046497,-2.469834,0.077924,0.078440,-0.082484,-0.806346


In [12]:
# SVR
svr = SVR()
params = {
    'kernel': ['rbf'],
    'gamma': [0.001, 0.01, 0.1, 1],
    'C': [0.01, 0.1, 1.0, 10.0]
}

model = GridSearchCV(
    estimator = svr,
    param_grid = params,
    scoring = ['neg_root_mean_squared_error', 'r2', 'explained_variance', 'neg_mean_squared_log_error', 'neg_mean_poisson_deviance'], 
    refit = 'neg_root_mean_squared_error',
    cv = 3,
    n_jobs = 3
)

model.fit(X, y)

GridSearchCV(cv=3, estimator=SVR(), n_jobs=3,
             param_grid={'C': [0.01, 0.1, 1.0, 10.0],
                         'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['rbf']},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error', 'r2', 'explained_variance',
                      'neg_mean_squared_log_error',
                      'neg_mean_poisson_deviance'])

In [13]:
d = pd.concat([pd.DataFrame(model.cv_results_["params"])
              ,pd.DataFrame(model.cv_results_['rank_test_neg_root_mean_squared_error'], columns=['Rank'])
              ,pd.DataFrame(model.cv_results_['mean_fit_time'], columns=['Fit Time'])
              ,pd.DataFrame(model.cv_results_['mean_score_time'], columns=['Score Time'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_root_mean_squared_error'], columns=['RMSE'])
              ,pd.DataFrame(model.cv_results_['mean_test_r2'], columns=['R2'])
              ,pd.DataFrame(model.cv_results_['mean_test_explained_variance'], columns=['EV'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_squared_log_error'], columns=['MSLE'])
              ,pd.DataFrame(model.cv_results_['mean_test_neg_mean_poisson_deviance'], columns=['MPD'])]
              ,axis=1)

d = d.sort_values(by = ['Rank', 'Score Time'], ascending=True)
d.to_csv('Results/18PCA-SVR.csv', decimal = ',', sep = ';', index = False)

d

,C,gamma,kernel,Rank,Fit Time,Score Time,RMSE,R2,EV,MSLE,MPD
10,1.00,0.100,rbf,1,11.358175,4.380431,-2.587592,-0.012347,0.046028,-0.086202,-0.902257
6,0.10,0.100,rbf,2,11.076937,4.390237,-2.597490,-0.019731,0.041767,-0.086414,-0.907439
13,10.00,0.010,rbf,3,11.893537,4.388578,-2.602102,-0.023491,0.033703,-0.087435,-0.913086
11,1.00,1.000,rbf,4,12.487499,4.380676,-2.604979,-0.025836,0.038854,-0.087365,-0.917701
14,10.00,0.100,rbf,5,14.293942,4.390812,-2.609050,-0.029835,0.021235,-0.089660,-0.926508
9,1.00,0.010,rbf,6,11.265345,4.387548,-2.610944,-0.030405,0.028336,-0.088095,-0.919928
7,0.10,1.000,rbf,7,11.206741,4.392011,-2.611949,-0.030742,0.036474,-0.087315,-0.919213
5,0.10,0.010,rbf,8,11.208626,4.387719,-2.617385,-0.035364,0.024064,-0.088325,-0.923662
2,0.01,0.100,rbf,9,11.131266,4.396372,-2.621355,-0.038249,0.024083,-0.088433,-0.926090
12,10.00,0.001,rbf,10,11.381633,4.358127,-2.621934,-0.039319,0.019216,-0.089527,-0.932389
